In [1]:
# Here we take care of paths.

from pathlib import Path
import os
print('Starting path:' + os.getcwd())
if os.getcwd()[-18:] == 'VESUVIUS_Challenge':
    pass
else:
    PATH = Path().resolve().parents[0]
    os.chdir(PATH)

# make sure you are in Paragraph_to_Tex folder
print('Current path:' + os.getcwd())

In [2]:
import torch
import monai
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from Data_Modules.Vesuvius_Tile_Dataset import Vesuvius_Tile_Datamodule
import matplotlib.patches as patches
from lit_models.UNET_TILE import UNET_TILE_lit
from monai.visualize import matshow3d
import einops
from pytorch_lightning.callbacks import ModelCheckpoint

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


2023-05-02 12:41:23,807 - Created a temporary directory at /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmp68xmbltu
2023-05-02 12:41:23,808 - Writing /var/folders/wc/60y8v25x3ns_jgsx6clbdb180000gn/T/tmp68xmbltu/_remote_module_non_scriptable.py


In [3]:
class CFG:
    
    train_fragment_id=[2,3]
    val_fragment_id=[1]
    batch_size = 32
    patch_size = 224
    z_dim = 16
    stride = patch_size // 2
    #comp_dataset_path = COMPETITION_DATA_DIR
    num_workers = 0
    on_gpu = False

In [4]:
dataset = Vesuvius_Tile_Datamodule(cfg=CFG)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

dataloader = iter(dataset.train_dataloader())
for i in range(1):
    # Get image and label from train data -- change number for different ones
    subvolumes, inklabels = next(dataloader)
    print('subvolume shape:',subvolumes.shape)
    print('inklabel shape:',inklabels.shape)
    for subvolume,  inklabel in zip(subvolumes, inklabels):
                fig, axes = plt.subplots(1, 2, figsize=(15, 5))
                for idx, image in enumerate((subvolume,  inklabel)):
                    if idx==0:
                        axes[idx].imshow(image[0])
                    else:
                         axes[idx].imshow(image.squeeze(0))

                plt.show()
    

plot_dataset = dataset.train_dataloader()
plot_count = 0
for i in range(1000):
    image, mask = plot_dataset[i]
    #data = transform(image=image, mask=mask)
    aug_image = image#.squeeze(0) # data['image']
    aug_mask = mask#.squeeze(0) #data['mask']
    print(image.shape, mask.shape)

    if mask.sum() == 0:
        continue

    fig, axes = plt.subplots(1, 4, figsize=(15, 8))
    axes[0].imshow(image[..., 0], cmap="gray")
    axes[1].imshow(mask, cmap="gray")
    #axes[2].imshow(aug_image[..., 0], cmap="gray")
    #axes[3].imshow(aug_mask, cmap="gray")
    
    #plt.savefig(CFG.figures_dir + f'aug_fold_{CFG.valid_id}_{plot_count}.png')

    plot_count += 1
    if plot_count == 5:
        break
    
    

In [5]:
lit_model = UNET_TILE_lit(
        use_wandb = False,
        z_dim = 16,
        patch_size = (224,224),
        sw_batch_size=8 ,
        eta_min = 1e-8,
        t_max = 250,
        max_epochs = 1000,
        weight_decay =  0.001,
        learning_rate = 0.0001,
        gamma = 0.85,)

#lit_model = lit_model.load_from_checkpoint('logs/FocalDICE_512_monai_unet_16_cont/lightning_logs/version_0/checkpoints/epoch=171-step=344.ckpt', 
#                                          #patch_size = (512,512),
#                                          learning_rate = 0.00001,
#                                           weight_decay =  .01,
#                                           eta_min = 1e-9,
#                                         sw_batch_size = 8)

In [6]:
checkpoint_callback = ModelCheckpoint(
    save_top_k=5,
    monitor="FBETA",
    mode="max",
    dirpath="logs/Full_16_768to1024_batch/",
    filename="FocalDice_768{epoch:02d}{FBETA:.2f}{recall:.2f}{precision:.2f}",
    save_last =True,
)


trainer = pl.Trainer(
        accelerator='cpu',
        #benchmark=True,
        max_epochs=100,
        check_val_every_n_epoch= 1,
        devices=1,
        #fast_dev_run=fast_dev_run,
        logger=pl.loggers.CSVLogger(save_dir='logs/Full_16_768to1024_batch/'),
        log_every_n_steps=1,
        default_root_dir = 'logs/Full_16_768to1024_batch/',
        #overfit_batches=1,
        #precision=16,
        accumulate_grad_batches=2, 
        callbacks=[checkpoint_callback],
        #resume_from_checkpoint ='logs/smp_unet_32_.5/lightning_logs/version_3/checkpoints/FocalDice_768epoch=123FBETA=0.30recall=0.77precision=0.26.ckpt'
        
        )





trainer.fit(lit_model, datamodule=dataset,
            #ckpt_path='logs/Full_16_768to1024_batch/FocalDice_768epoch=151FBETA=0.41recall=0.56precision=0.39.ckpt'
           )
# resume_from_checkpoint = 
#ckpt_path='logs/unet_smp-epoch=102-val_loss=0.00.ckpt'
# ckpt_path='logs/Eff_monai_32z/lightning_logs/version_0/checkpoints/epoch=19-step=40.ckp'

2023-05-02 12:41:47,262 - GPU available: True (mps), used: False
2023-05-02 12:41:47,263 - TPU available: False, using: 0 TPU cores
2023-05-02 12:41:47,263 - IPU available: False, using: 0 IPUs
2023-05-02 12:41:47,263 - HPU available: False, using: 0 HPUs
Adjusting learning rate of group 0 to 1.0000e-04.
2023-05-02 12:41:47,371 - 
   | Name                 | Type                  | Params
----------------------------------------------------------------
0  | metrics              | ModuleDict            | 0     
1  | model                | UNet                  | 6.5 M 
2  | loss                 | MaskedLoss            | 0     
3  | mine_focal           | FocalLoss             | 0     
4  | weighted_bce_loss    | BCEWithLogitsLoss     | 0     
5  | loss_dice            | DiceLoss              | 0     
6  | loss_tversky         | TverskyLoss           | 0     
7  | loss_bce             | SoftBCEWithLogitsLoss | 0     
8  | loss_focal           | FocalLoss             | 0     
9  | dicelos

/Users/gregory/anaconda3/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/Users/gregory/anaconda3/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /Users/gregory/PROJECT_ML/VESUVIUS_Challenge/logs/Full_16_768to1024_batch exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
/Users/gregory/anaconda3/lib/python3.10/site-packages/lightning_fabric/loggers/csv_logs.py:188: UserWarning: Experiment logs directory logs/Full_16_768to1024_batch/lightning_logs/version_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  rank_zero_warn(


Sanity Checking: 0it [00:00, ?it/s]

/Users/gregory/anaconda3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/gregory/anaconda3/lib/python3.10/site-packages/monai/losses/spatial_mask.py:51: UserWarning: No mask value specified for the MaskedLoss.
  warnings.warn("No mask value specified for the MaskedLoss.")


ValueError: The `target` has to be an integer tensor.